In [12]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, parent_dir)

from CUSIP_Curve import CUSIP_Curve

from dotenv import dotenv_values
env_path = os.path.join(parent_dir, ".env")
config = dotenv_values(env_path)

from datetime import datetime

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Fetching with CUSIP_Curve:
- Timeseries by CUSIP
- CT EOD, Bid, Offer, Mid Yields sourced from FedInvest
- CMT Yields from FRED (Fed) or Treasury - Both sources likely pull from the same/use same par curve build (Treasury's monotone convex interpolation performed on forward rates from bids around 3:30 PM EST by FRBNY)
- Pull All Active CUSIPs given date

In [11]:
cusip_curve_builder = CUSIP_Curve(use_ust_issue_date=True, no_logs_plz=True, fred_api_key=config["FRED_API_KEY"])

# Fetching Timeseries data by CUSIPs:

In [13]:
cusip_dict_df = cusip_curve_builder.cusips_timeseries(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 8, 20),
    cusips=["912810UA4", "912810UC0", "912810ES3"],
)
cusip_dict_df

{'912810UA4':          Date  bid_price  offer_price   mid_price  eod_price  bid_yield  \
 0  2024-05-15  101.68750   101.687500  101.687500  101.78125   4.521655   
 1  2024-05-16  102.00000   102.015625  102.007812  101.71875   4.502751   
 2  2024-05-17  101.31250   101.312500  101.312500  101.03125   4.544356   
 3  2024-05-20  100.68750   100.703125  100.695312  100.78125   4.582534   
 4  2024-05-21  101.15625   101.156250  101.156250  101.18750   4.553849   
 ..        ...        ...          ...         ...        ...        ...   
 61 2024-08-13  107.75000   107.765625  107.757812  107.84375   4.167710   
 62 2024-08-14  108.62500   108.640625  108.632812  108.56250   4.119046   
 63 2024-08-15  107.21875   107.234375  107.226562  107.50000   4.197442   
 64 2024-08-16  107.56250   107.578125  107.570312  107.90625   4.178109   
 65 2024-08-19  108.53125   108.562500  108.546875  108.62500   4.124122   
 
     offer_yield  eod_yield  
 0      4.521655   4.515987  
 1      4.501

# Fetching CT Yields:
- CT Yields: Market observed yields of on-the-runs (most recent issue for respective bill, note, bonds) 
- Default yields are EOD price markings according to FedInvest converted to YTM with rateslibs
- Able to enable bid, offer, mide side yields as of ~3:30 PM CST
- All tenors are returned by default, able to pass in specific tenor strings

In [15]:
ct_yields_df = cusip_curve_builder.get_historical_ct_yields(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 8, 19),
    # use_bid_side=True,
    # use_offer_side=True,
    # use_mid_side=True,
    # tenors=[
    #     "8-Week", 
    #     "17-Week",
    #     "26-Week",
    #     "52-Week",
    #     "2-Year",
    #     "3-Year",
    #     "5-Year",
    #     "7-Year",
    #     "10-Year",
    #     "20-Year",
    #     "30-Year",
    # ],
)
ct_yields_df

,Date,8-Week,17-Week,26-Week,52-Week,2-Year,3-Year,5-Year,7-Year,10-Year,20-Year,30-Year
0,2024-01-02,NaN,5.414579,5.266761,4.812030,4.332836,4.101289,3.930939,3.951272,3.944279,4.248016,4.083422
1,2024-01-03,NaN,5.403556,5.277076,4.822615,4.332881,4.078369,3.896061,3.914976,3.906654,4.209880,4.053157
2,2024-01-04,NaN,5.403156,5.250537,4.855018,4.383067,4.145664,3.973300,3.993184,3.988959,4.295227,4.137584
3,2024-01-05,NaN,5.391337,5.238747,4.854037,4.399899,4.179603,4.015587,4.040325,4.049422,4.363582,4.208147
4,2024-01-08,NaN,5.390940,5.227726,4.831904,4.366566,4.122418,3.966455,3.993320,4.011403,4.322405,4.176983
...,...,...,...,...,...,...,...,...,...,...,...,...
151,2024-08-13,NaN,5.170481,4.984385,4.407569,3.939168,3.768811,3.679929,3.734074,3.847467,4.243567,4.162472
152,2024-08-14,NaN,5.201923,5.005497,4.428822,3.955202,3.768255,3.672857,3.718719,3.828465,4.209461,4.122502
153,2024-08-15,NaN,5.212155,5.050511,4.536665,4.087457,3.895155,3.790051,3.825178,3.916866,4.277710,4.176494
154,2024-08-16,NaN,5.189838,5.027992,4.503012,4.070216,3.872829,3.769089,3.804630,3.890165,4.259370,4.152878


# Fetching CMT Yields:

In [16]:
cmt_yields_fred_df = cusip_curve_builder.get_historical_cmt_yields(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 8, 19),
)
cmt_yields_fred_df

Fetching from FRED...


,Date,13-Week,26-Week,52-Week,2-Year,3-Year,5-Year,7-Year,10-Year,20-Year,30-Year
0,2024-01-02,5.22,5.04,4.80,4.33,4.09,3.93,3.95,3.95,4.25,4.08
1,2024-01-03,5.24,5.06,4.81,4.33,4.07,3.90,3.92,3.91,4.21,4.05
2,2024-01-04,5.24,5.06,4.85,4.38,4.14,3.97,3.99,3.99,4.30,4.13
3,2024-01-05,5.24,5.05,4.84,4.40,4.17,4.02,4.04,4.05,4.37,4.21
4,2024-01-08,5.25,5.04,4.82,4.36,4.11,3.97,3.99,4.01,4.33,4.17
...,...,...,...,...,...,...,...,...,...,...,...
154,2024-08-13,5.06,4.75,4.40,3.93,3.75,3.68,3.74,3.85,4.25,4.16
155,2024-08-14,5.07,4.77,4.42,3.94,3.76,3.67,3.72,3.83,4.22,4.12
156,2024-08-15,5.09,4.82,4.52,4.08,3.90,3.79,3.83,3.92,4.28,4.18
157,2024-08-16,5.08,4.80,4.49,4.06,3.87,3.77,3.81,3.89,4.26,4.15


## Advantage of using treaaury.gov source:
- Download xlsx
- Able to apply treasury long-term rates Extrapolation Factors: [UST's Daily Treasury Long-Term Rates](https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_long_term_rate&field_tdr_date_value_month=202408)

In [35]:
cmt_yields_ust_df = cusip_curve_builder.get_historical_cmt_yields(
    start_date=datetime(2000, 1, 1),
    end_date=datetime(2024, 8, 19),
    use_treasury_par=True, # must enable
    treasury_data_dir=r"C:\Users\chris\Curvy-CUSIPs\data\treasury", # change to path you want
    download_treasury_par_yields=True,
    apply_long_term_extrapolation_factor=True, # Only enable if start_date is older than February 9, 2006
)
cmt_yields_ust_df

Fetching from treasury.gov...


,Date,4-Week,8-Week,13-Week,17-Week,26-Week,52-Week,2-Year,3-Year,5-Year,7-Year,10-Year,20-Year,30-Year
0,2001-01-02,NaN,NaN,5.87,NaN,5.58,5.11,4.87,4.82,4.76,4.97,4.92,5.46,5.35
1,2001-01-03,NaN,NaN,5.69,NaN,5.44,5.04,4.92,4.92,4.94,5.18,5.14,5.62,5.49
2,2001-01-04,NaN,NaN,5.37,NaN,5.20,4.82,4.77,4.78,4.82,5.07,5.03,5.56,5.44
3,2001-01-05,NaN,NaN,5.12,NaN,4.98,4.60,4.56,4.57,4.66,4.93,4.93,5.50,5.41
4,2001-01-08,NaN,NaN,5.19,NaN,5.03,4.61,4.54,4.55,4.65,4.94,4.94,5.52,5.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5907,2024-08-13,5.48,5.39,5.32,5.18,4.97,4.40,3.93,3.75,3.68,3.74,3.85,4.25,4.16
5908,2024-08-14,5.49,5.39,5.32,5.21,5.00,4.42,3.94,3.76,3.67,3.72,3.83,4.22,4.12
5909,2024-08-15,5.53,5.40,5.34,5.22,5.04,4.52,4.08,3.90,3.79,3.83,3.92,4.28,4.18
5910,2024-08-16,5.53,5.40,5.33,5.21,5.02,4.49,4.06,3.87,3.77,3.81,3.89,4.26,4.15


# Pulling all Active CUSIPs given date (Bills, Notes, Bonds):
- Able to pull in respective CUSIP's auction results
- Latest (wrt paseed in as of date) STRIPping activity
- Latest (wrt pass in as of date) SOMA Holdings

In [39]:
curve_set_df = cusip_curve_builder.build_curve_set(
    as_of_date=datetime(2024, 8, 19),
    # include_auction_results=True,
    # include_soma_holdings=True,
    # include_stripping_activity=True,
    sorted=True, # recommended to enable this
    use_github=True, # recommended to enable this
)
curve_set_df

,cusip,security_type,auction_date,issue_date,maturity_date,time_to_maturity,int_rate,high_investment_rate,is_on_the_run,label,security_term,original_security_term,offer_yield,bid_yield,eod_yield,mid_yield
0,912797KY2,Bill,2024-07-18,2024-07-23,2024-08-20,0.002740,NaN,5.365000,False,Aug 24s,4-Week,17-Week,NaN,NaN,NaN,NaN
1,912797KZ9,Bill,2024-07-25,2024-07-30,2024-08-27,0.021918,NaN,5.381000,False,Aug 24s,4-Week,17-Week,6.145361,6.151317,5.371803,6.148339
2,912797LA3,Bill,2024-08-01,2024-08-06,2024-09-03,0.041096,NaN,5.381000,False,Sep 24s,4-Week,17-Week,5.822992,5.828631,5.434437,5.825811
3,912797LG0,Bill,2024-08-08,2024-08-13,2024-09-10,0.060274,NaN,5.381000,False,Sep 24s,4-Week,17-Week,5.700234,5.700234,5.447776,5.700234
4,912797LH8,Bill,2024-07-18,2024-07-23,2024-09-17,0.079452,NaN,5.377000,False,Sep 24s,8-Week,17-Week,5.659341,5.670280,5.461146,5.664811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,912810TT5,Bond,2023-10-12,2023-10-16,2053-08-15,29.008219,4.125,NaN,False,Aug 53s,29-Year 10-Month,30-Year,4.136123,4.137987,4.134261,4.137055
389,912810TV0,Bond,2024-01-11,2024-01-16,2053-11-15,29.260274,4.750,NaN,False,Nov 53s,29-Year 10-Month,30-Year,4.118437,4.119297,4.114139,4.118867
390,912810TX6,Bond,2024-04-11,2024-04-15,2054-02-15,29.512329,4.250,NaN,False,Feb 54s,29-Year 10-Month,30-Year,4.131089,4.131998,4.128364,4.131543
391,912810UA4,Bond,2024-07-11,2024-07-15,2054-05-15,29.756164,4.625,NaN,False,May 54s,29-Year 10-Month,30-Year,4.122392,4.124122,4.118935,4.123257
